Read the raw driver data from `/mnt/cis442f-data/duocar/raw/drivers` into a Spark DataFrame.

In [0]:
drivers = spark.read.csv("/mnt/cis442f-data/duocar/raw/drivers", header=True, inferSchema=True)

How young is the youngest driver?  How old is the oldest driver?

In [0]:
from pyspark.sql.functions import to_date, current_date, months_between, floor
drivers \
  .select(to_date("birth_date", "yyyy-MM-dd").alias("birth_date_fixed"), current_date().alias("today")) \
  .withColumn("age", floor(months_between("today", "birth_date_fixed") / 12)) \
  .sort("birth_date_fixed")\
  .show(1)
drivers \
  .select(to_date("birth_date", "yyyy-MM-dd").alias("birth_date_fixed"), current_date().alias("today")) \
  .withColumn("age", floor(months_between("today", "birth_date_fixed") / 12)) \
  .sort("birth_date_fixed", ascending=False)\
  .show(1)

+----------------+----------+---+
|birth_date_fixed|     today|age|
+----------------+----------+---+
|      1947-09-26|2022-04-14| 74|
+----------------+----------+---+
only showing top 1 row

+----------------+----------+---+
|birth_date_fixed|     today|age|
+----------------+----------+---+
|      1999-03-22|2022-04-14| 23|
+----------------+----------+---+
only showing top 1 row



How many female drivers does DuoCar have?  How many non-white, female drivers?

In [0]:
print("Female drivers")
print(drivers.where(drivers.sex == "female").count())
print("non-white, female drivers")
print(drivers.filter(drivers.sex == "female").where(drivers. ethnicity != "White").count())

Female drivers
108
non-white, female drivers
9


Create a new DataFrame without any personally identifiable information (PII) 
- Create a new column "birth_year"
- PII fields to remove: "first_name", "last_name", "home_block" , "home_lat", "home_lon", "birth_date"

In [0]:
from pyspark.sql.functions import year
from pyspark.sql.functions import to_date
drivers2 = spark.read.csv("/mnt/cis442f-data/duocar/raw/drivers", header=True, inferSchema=True)
drivers2 \
  .withColumn("birth_year", year(drivers2.birth_date)) \
  .drop("first_name", "last_name", "home_block" , "home_lat", "home_lon", "birth_date") \
  .show(15)

+------------+-------------------+------+---------+-------+------------+-------------+------------+-------------+-------------+------------+-------------+-----+-----+----------+
|          id|         start_date|   sex|ethnicity|student|vehicle_make|vehicle_model|vehicle_year|vehicle_color|vehicle_grand|vehicle_noir|vehicle_elite|rides|stars|birth_year|
+------------+-------------------+------+---------+-------+------------+-------------+------------+-------------+-------------+------------+-------------+-----+-----+----------+
|220200000007|2017-01-01 00:00:00|  male|    White|      1|   Chevrolet|        Cruze|        2013|         gray|            0|           0|            0|   89|  398|      1996|
|220200000043|2017-01-02 00:00:00|  male|    White|      0|         GMC|     Yukon XL|        2016|        black|            1|           1|            1|  239| 1108|      1993|
|220200000053|2017-01-02 00:00:00|  male|    White|      0|    INFINITI|         EX35|        2008|          r

Read the raw ride data from `/mnt/cis442f-data/duocar/raw/rides` into a Spark DataFrame.  Inspect the  `service` column.  Replace the missing values with "Car" for standard DuoCar service.

In [0]:
ride = spark.read.csv("/mnt/cis442f-data/duocar/raw/rides", header=True, inferSchema=True)
ride.fillna("Car", ["service"]).show(25)

+---+------------+------------+-------------------+----------+-------+----------+----------+---------+----------+--------+--------+---------+-----------+
| id|   driver_id|    rider_id|          date_time|utc_offset|service|origin_lat|origin_lon| dest_lat|  dest_lon|distance|duration|cancelled|star_rating|
+---+------------+------------+-------------------+----------+-------+----------+----------+---------+----------+--------+--------+---------+-----------+
|  1|220200000214|220200000084|2017-02-01 00:14:00|        -6|    Car| 46.850956|-96.902849| 46.86005|-96.825442|   10123|     729|        0|          5|
|  2|220200000107|220200000462|2017-02-01 00:36:00|        -6|    Car| 46.900432|-96.765807|46.840588|-96.868087|   16043|    1299|        0|          5|
|  3|220200000214|220200000489|2017-02-01 02:26:00|        -6|   Noir| 46.868382|-96.902718|46.815272|-96.862056|    9362|     736|        0|          5|
|  4|220200000067|220200000057|2017-02-01 03:00:00|        -6|    Car| 46.90

Create a Dataframe that:

- Convert the rides.driver_id column to a string column.
- Extract the year from the rides.date_time column (hint: you can use the year function)
- Convert rides.duration from seconds to minutes.
- Convert the rides.cancelled column to a boolean column.
- Convert the rides.star_rating column to a double column.

In [0]:
rides = spark.read.csv("/mnt/cis442f-data/duocar/raw/rides/", header=True, inferSchema=True)
from pyspark.sql.functions import format_string, year, col, round
rides = rides\
  .withColumn("driver_id", rides.driver_id.cast("string"))\
  .withColumn("date_time", year(rides.date_time))\
  .withColumn("duration", round(rides.duration/60))\
  .withColumn("cancelled", rides.cancelled.cast("boolean"))\
  .withColumn("star_rating", rides.star_rating.cast("double"))

rides.show(5)
rides.printSchema()

+---+------------+------------+---------+----------+-------+----------+----------+---------+----------+--------+--------+---------+-----------+
| id|   driver_id|    rider_id|date_time|utc_offset|service|origin_lat|origin_lon| dest_lat|  dest_lon|distance|duration|cancelled|star_rating|
+---+------------+------------+---------+----------+-------+----------+----------+---------+----------+--------+--------+---------+-----------+
|  1|220200000214|220200000084|     2017|        -6|   null| 46.850956|-96.902849| 46.86005|-96.825442|   10123|    12.0|    false|        5.0|
|  2|220200000107|220200000462|     2017|        -6|   null| 46.900432|-96.765807|46.840588|-96.868087|   16043|    22.0|    false|        5.0|
|  3|220200000214|220200000489|     2017|        -6|   Noir| 46.868382|-96.902718|46.815272|-96.862056|    9362|    12.0|    false|        5.0|
|  4|220200000067|220200000057|     2017|        -6|   null| 46.908567|-96.905391| 46.90438|-96.793999|    9060|    13.0|    false|     